In [ ]:
v1 = vector([1,1,1,1,1,1,1,1])
v2 = vector([1,1,1,1,-1,-1,-1,-1])
v3 = vector([1,1,-1,-1,0,0,0,0])
v4 = vector([0,0,0,0,1,1,-1,-1])
v5 = vector([1,-1,0,0,0,0,0,0])
v6 = vector([0,0,1,-1,0,0,0,0])
v7 = vector([0,0,0,0,1,-1,0,0])
v8 = vector([0,0,0,0,0,0,1,-1])

A = matrix([v1,v2,v3,v4,v5,v6,v7,v8]).transpose()
#Initiating the matrix

In [ ]:
A.rref()

In [ ]:
v = vector([54,198,43,168,208,117,8,181])

va = A.inverse()*v

Aaug = A.augment(v,subdivide = True)

Aaug.rref().column(-1)

In [ ]:
from sage.plot.matrix_plot import MatrixPlot

In [ ]:
def threshold_vectorizer(threshold, vectr):
    dummy = vector([i for i in vectr])
    for a in range(len(dummy)):
        if abs(dummy[a]) < threshold:
            dummy[a] = 0
    return dummy

In [ ]:
v1 = threshold_vectorizer(10, va)
v2 = threshold_vectorizer(50, va)
v3 = threshold_vectorizer(100, va)


threshold_vectors = [va, v1,v2,v3]
v1, v2, v3

threshold_vectors_in_basis = [A*i for i in threshold_vectors]

for vec in range(len(threshold_vectors_in_basis)):
    print(f"""
    The corresponding vector in the new basis  is
    {threshold_vectors[vec].n(digits = 6)} 
    and after conversion back to the standard basis it becomes 
    {threshold_vectors_in_basis[vec].n(digits = 6)}""")

In [ ]:
def vector_map(vector):
    return matrix_plot(matrix([vector]), cmap = 'Greys', vmin = 0, vmax = 256)

In [ ]:
thresholds = [0,10,50,100]
for a in range(len(threshold_vectors)):
    print(f'''
    With the threshold {thresholds[a]}, you see the following compression''')
    show(vector_map(threshold_vectors_in_basis[a]))

In [ ]:
thresholds = [a*5 for a in range(1,21)]
threshold_vectors = [A*(threshold_vectorizer(i,va)) for i in thresholds]

for a in range(len(threshold_vectors)):
    print(f'''
    With the threshold {thresholds[a]}, you see the following compression''')
    show(vector_map(threshold_vectors[a]))

In [ ]:
from PIL import Image
from numpy import asarray
  
  
# load the image and convert into 
# numpy array
img = Image.open('Hafiz.png')
numpydata = asarray(img)
  
# data
print(numpydata.shape)

# Below is the way of creating Pillow 
# image from our numpyarray
pilImage = Image.fromarray(numpydata)
print(type(pilImage))
  
# Let us check  image details
print(pilImage.mode)
print(pilImage.size)

In [ ]:
def threshold_vectorizer(threshold, vectr):
    dummy = vector([i for i in vectr])
    for a in range(len(dummy)):
        if abs(dummy[a]) < threshold:
            dummy[a] = 0
    return dummy

In [ ]:
#Creating a haar wavelet basis of size n

def wavelet(n):
    '''
    Creates haar wavelet basis vectors of size n'''
    vectors = []
    for a in range(log(n,2)+1): #The different regions of 1s and -1s, which are equal to the log with base 2 of n plus 1
        if a == 0:
            vectors.append(vector([1]*n)) #1st vector which is always 1s
        elif a == 1: #2nd vector which is always half 1s and half -1s
            a = [1]*int(n/2)
            b = [-1]*int(n/2)
            c = a + b
            vectors.append(vector(c))
        else: #generalizing it out to the other bunches of vectors 
            
            
            current_pos = 0 #Indicates where you'll be adding the current 1,-1 pair
            for b in range(2**(a-1)):
                current_a = [1]*int(n/(2^(a)))
                current_b = [-1]*int(n/(2^(a)))
                current_c = current_a + current_b
                
                
                hector = []
                for i in range(2**(a-1)):
                    if i == current_pos:
                        hector += current_c
                    else:
                        hector += [0]*int(len(current_c))
                current_pos +=1
                vectors.append(vector(hector))
    
                #This block of code makes the pair, adds it iteratively into the right position and updates the position for
        #the next part to get added
    return vectors     

In [ ]:
import numpy as np
from PIL import Image

mario = Image.open('1.png').convert('L')

mario
image_in_greyscale = np.asarray(mario)
n = (len(image_in_greyscale))^2
M = matrix(wavelet(n))

In [ ]:
matrix_plot(image_in_greyscale)

In [ ]:
def compressor(threshold,image):
    
    image_in_greyscale = np.asarray(image)
    image_in_greyscale_inverted = [255 - i for i in image_in_greyscale.ravel()]
    width = len(image_in_greyscale)
    n = (width)^2
    haar = matrix(wavelet(n))

    Mt = haar*threshold_vectorizer(threshold, M.inverse() * vector(image_in_greyscale_inverted))

    qwer = matrix(Mt)
    qwer32 = np.reshape(qwer, (width,width))

    return qwer32

In [ ]:
thresholds = [0,5,15,35]
image_names = ['oogway','linear','logo']


oogway = Image.open('oogway.png').convert('L')
linear = Image.open('linear.png').convert('L')
logo = Image.open('logo.png').convert('L')

images = [oogway,linear,logo]

for image in range(len(images)):
    for threshold in range(len(thresholds)):
        print(f"With a threshold of {thresholds[threshold]}, the compressed image for {image_names[image]} looks like ")
        compressed = compressor(thresholds[threshold],images[image])
        show(matrix_plot(compressed))
        
        inverted_compressed = [255 - i for i in compressed.ravel()]
        inverted_matrix = np.reshape(inverted_compressed, (32,32))
        
        image1 = Image.fromarray(inverted_matrix.astype(np.uint8))
        image1 = image1.convert('RGB')
        image1.save(f"{image_names[image]}{thresholds[threshold]}.png")

In [ ]:
image = Image.fromarray(qwer32)
image = image.convert('RGB')
image.save('wow.png')

In [ ]:
qwer32.shape

In [7]:
MR = matrix([[-1,0,0,1],[1,-1,0,0],[0,1,-1,0],[0,0,1,-1]]).transpose()
MC = matrix([[-1,-1,0],[1,0,1],[0,1,-1]]).transpose()
MK = matrix([[-1,-1,-1,0,0,0],[1,0,0,-1,-1,0],[0,1,0,1,0,-1],[0,0,1,0,1,1]]).transpose()

MC,MK

(
            [-1  1  0  0]
            [-1  0  1  0]
            [-1  0  0  1]
[-1  1  0]  [ 0 -1  1  0]
[-1  0  1]  [ 0 -1  0  1]
[ 0  1 -1], [ 0  0 -1  1]
)

In [12]:
var('x1,x2,x3,x4')

MR*vector([x1,x2,x3,x4])

(-x1 + x2, -x2 + x3, -x3 + x4, x1 - x4)

In [13]:
MC*vector([x1,x2,x3])

(-x1 + x2, -x1 + x3, x2 - x3)

In [14]:
MK*vector([x1,x2,x3,x4])

(-x1 + x2, -x1 + x3, -x1 + x4, -x2 + x3, -x2 + x4, -x3 + x4)

In [15]:
MC.rref()

[ 1  0 -1]
[ 0  1 -1]
[ 0  0  0]

In [16]:
var('y1,y2,y3,y4,y5,y6')

MR.transpose()*vector([y1,y2,y3,y4])

(-y1 + y4, y1 - y2, y2 - y3, y3 - y4)

In [17]:
MC.transpose()*vector([y1,y2,y3])

(-y1 - y2, y1 + y3, y2 - y3)

In [19]:
MKflows = MK.transpose()*vector([y1,y2,y3,y4,y5,y6])

In [29]:
var('y1,y2,y6')

solve([-y1-y2+1 == 0, y1 - 10 == 0, y2 + 10 - y6 == 0, -1 + y6 == 0],y1,y2,y6)

[[y1 == 10, y2 == -9, y6 == 1]]

In [23]:
MKflows

(-y1 - y2 - y3, y1 - y4 - y5, y2 + y4 - y6, y3 + y5 + y6)

In [31]:
Md = matrix([[-1,1,0,0,0,0,0],[0,-1,1,0,0,0,0],[0,0,-1,1,0,0,0],[0,0,0,1,-1,0,0],[0,0,0,0,1,-1,0],\
           [0,0,-1,0,0,1,0],[0,0,1,0,0,0,-1],[1,0,0,0,0,0,-1]])

In [33]:
Md, Md.rref()

(
[-1  1  0  0  0  0  0]  [ 1  0  0  0  0  0 -1]
[ 0 -1  1  0  0  0  0]  [ 0  1  0  0  0  0 -1]
[ 0  0 -1  1  0  0  0]  [ 0  0  1  0  0  0 -1]
[ 0  0  0  1 -1  0  0]  [ 0  0  0  1  0  0 -1]
[ 0  0  0  0  1 -1  0]  [ 0  0  0  0  1  0 -1]
[ 0  0 -1  0  0  1  0]  [ 0  0  0  0  0  1 -1]
[ 0  0  1  0  0  0 -1]  [ 0  0  0  0  0  0  0]
[ 1  0  0  0  0  0 -1], [ 0  0  0  0  0  0  0]
)